In [51]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from text_processing import preprocess_text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_excel('../data.xlsx', nrows=3800)

In [52]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Tokenize and preprocess the text
texts = df['text'].astype(str).apply(preprocess_text).values

In [53]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [54]:
labels = df['id'].values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

with open('encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [55]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

In [56]:
embedding_dim = 90
LSTM1 = 80
LSTM2 = 90
optimalizer = 'adam'
batch_size = 2
epochs = 8

In [57]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBEDDING_DIM = embedding_dim
MAX_LEN = len(data[0])
OUTPUT_CLASSES = len(y_train[0])
print(OUTPUT_CLASSES)

33


Model

In [58]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(LSTM(LSTM1, return_sequences=True))  # Změna na True pro více vrstev LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(LSTM(LSTM2))  # Další vrstva LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(Dense(OUTPUT_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimalizer,
              metrics=['accuracy'])


In [59]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,  # Zvýšení počtu epoch
          validation_data=(X_test, y_test))

Epoch 1/8
1328/1328 [==============================] - 79s 54ms/step - loss: 2.5549 - accuracy: 0.2861 - val_loss: 1.0869 - val_accuracy: 0.7243
Epoch 2/8
1328/1328 [==============================] - 72s 54ms/step - loss: 0.7082 - accuracy: 0.7922 - val_loss: 0.4212 - val_accuracy: 0.8841
Epoch 3/8
1328/1328 [==============================] - 85s 64ms/step - loss: 0.2946 - accuracy: 0.9179 - val_loss: 0.3218 - val_accuracy: 0.9104
Epoch 4/8
1328/1328 [==============================] - 87s 66ms/step - loss: 0.2010 - accuracy: 0.9413 - val_loss: 0.3077 - val_accuracy: 0.9104
Epoch 5/8
1328/1328 [==============================] - 89s 67ms/step - loss: 0.1365 - accuracy: 0.9623 - val_loss: 0.3242 - val_accuracy: 0.9175
Epoch 6/8
1328/1328 [==============================] - 84s 64ms/step - loss: 0.1157 - accuracy: 0.9639 - val_loss: 0.4109 - val_accuracy: 0.9052
Epoch 7/8
1328/1328 [==============================] - 72s 54ms/step - loss: 0.0839 - accuracy: 0.9725 - val_loss: 0.3123 - val_ac

In [110]:
model.save('text_model')

INFO:tensorflow:Assets written to: text_model\assets


INFO:tensorflow:Assets written to: text_model\assets


In [7]:
from keras import models
from keras.models import load_model
model = load_model('../text_model')

In [79]:
class_names = ["lukostřelba", "baseball", "basketbal", "kulečník", "bmx", "bowling", "box", "jízda na býku", "roztleskávání", "curling", "šerm", "krasobruslení", "fotbal", "závody formule 1", "golf", "skok do výšky", "hokej", "dostihy", "hydroplánové závody", "judo", "motocyklové závody", "pole dance", "rugby", "skoky na lyžích", "snowboarding", "rychlobruslení", "surfování", "plavání", "stolní tenis", "tenis", "dráhové kolo", "volejbal", "vzpírání"]
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)
string = preprocess_text('rugby se hraje na trávě')
sequences = tokenizer.texts_to_sequences([string])
data = pad_sequences(sequences, maxlen=28)
predictions = model.predict(data)
predicted_labels = predictions.argmax(axis=-1)
predicted_labels = encoder.inverse_transform(predicted_labels)
predicted_confidence = predictions.max(axis=-1)
print(class_names[predicted_labels[0]])
print(f"jistota: {predicted_confidence[0]*100} %")

1/1 [==============================] - 0s 72ms/step
surfování
jistota: 23.122380673885345 %
